In [106]:
import numpy as np
from scipy.stats import norm, bernoulli, multivariate_normal

In [488]:
rho = 1e-4
c = 100000

In [489]:
def sample_gammas(gammas_probs):
    return [1 if np.random.rand() < p else 0 for p in gammas_probs]

def var_w(gamma): # variance of w
    assert gamma in [0, 1]
    return rho if gamma == 0 else c * rho

def sample_weights(gammas):
    return [np.random.normal(0, var_w(gamma)) for gamma in gammas]

def gaussian(x, mu, var):
    coef = 1 / np.sqrt(var * 2 * np.pi) 
    exp = np.exp(-0.5 * ((x - mu) ** 2)/var)
    return coef * exp

def prob_w(w, gamma):
    return gaussian(w, 0, var_w(gamma))

def prob_gamma(gamma):
    return rho if gamma == 0 else c * rho

In [490]:
def gamma_post(k, gammas_probs, gammas, ws):
    p0 = (1 - gammas_probs[k]) * prob_w(ws[k], 0)
    p1 = gammas_probs[k] * prob_w(ws[k], 1)
    return p1 / (p0 + p1)

def w_post(X, y, gammas, noise): # p(w | stuff)
    diag = [1 / ((c ** gamma) * rho) for gamma in gammas]
    R_gamma_inv = np.diag(diag)
    xtx = X.T @ X
    mean = np.linalg.inv(noise * R_gamma_inv + xtx) @ X.T @ y
    var = np.linalg.inv(R_gamma_inv + xtx / noise)

    return np.random.multivariate_normal(mean, var)

In [531]:
def gibbs(X, y, n, noise):
    gammas_probs = np.repeat(0.1, X.shape[1])
    gammas = sample_gammas(gammas_probs)
    ws = w_post(X, y, gammas, noise)
    
    integral = np.zeros(X.shape[1])
    for i in range(n):
        ws = w_post(X, y, gammas, noise)
        for i in range(len(gammas_probs)):
            gammas_probs[i] = gamma_post(i, gammas_probs, gammas, ws)
        gammas = sample_gammas(gammas_probs)
        
        w_probs = np.zeros(X.shape[1])
        gamma_prior = np.zeros(X.shape[1])
        for i in range(len(w_probs)):
            w_probs[i] = prob_w(ws[i], gammas[i])
            gamma_prior[i] = prob_gamma(gammas[i])
            
        integral += ws * gammas * w_probs * gammas_probs
    return ws, gammas, integral

In [532]:
d = 10
s = 5
n = 1000
noise = 0.1
X = np.random.uniform(size=(n, d))
ws = np.zeros(d)
indices = np.arange(d)
np.random.shuffle(indices)
ws[indices[:s]] = 1
print(ws)
y = np.dot(X, ws) + np.random.normal(0, noise)

[0. 0. 1. 1. 1. 1. 1. 0. 0. 0.]


In [533]:
runs = 100
ws, gammas, integral = gibbs(X, y, runs, noise)
ws

array([-0.00989339, -0.03716892,  1.02333738,  0.95278299,  1.01901316,
        1.00566242,  1.02220576,  0.00549711,  0.09761195,  0.02113016])

In [534]:
(integral / runs)

array([0.00141656, 0.00147013, 0.12089441, 0.1200156 , 0.11998808,
       0.12026965, 0.12010293, 0.00150982, 0.00224957, 0.00338816])

In [535]:
np.array(gammas)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [524]:
np.multiply(ws, gammas)[indices[:s]]

array([1.00019966, 1.02427643, 0.9976252 , 1.06340773, 0.98652045])